In [1]:
import itertools
import random
import numpy as np
import pandas as pd
import seaborn as sns
import cvxpy as cp
from tqdm import tqdm

from scipy.spatial import distance
from cvxpy.error import SolverError
from cvxpy import ECOS, SCS
import seaborn as sns
import ot
from pgmpy import inference
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance

import scipy.optimize as optimize
from scipy.spatial.distance import cdist
from src.examples import smokingmodels as sm
from scipy.spatial.distance import squareform,pdist
from scipy.stats import wasserstein_distance

from scipy.optimize import linprog
from scipy import stats
from IPython.utils import io
import warnings

from scipy.stats import entropy
from scipy.spatial import distance
from scipy.special import kl_div
import pandas as pd
import cvxpy as cp
from cvxpy.error import SolverError
from cvxpy import ECOS, SCS
from collections import Counter
import seaborn as sns
from pgmpy import inference
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance
import scipy.optimize as optimize
from scipy.spatial.distance import cdist
from src.examples import smokingmodels as sm
from scipy.spatial.distance import squareform,pdist
from scipy.optimize import linprog
from scipy import stats
from IPython.utils import io
import joblib
from sklearn.linear_model import Lasso,LinearRegression

import joblib
import modularized_utils as ut
import abstraction_metrics as ams
import matplotlib.pylab as pl

import cost_functions_utils as cts

import get_results
import params

np.random.seed(0)

warnings.filterwarnings(action='ignore')
np.set_printoptions(precision=4,suppress=True)

In [2]:
experiments = {
                1: 'synth1',
                2: 'synth2',
                3: 'synth1inv',
                4: 'synthX1',
                5: 'lucas',
                6: 'little_lucas',
                7: 'battery_discrete',
                8: 'battery_continuous'
              }

In [3]:
def create_comb(a, b, c):
    return str(a)+'-'+str(b)+'-'+str(c)

def parse_comb(input_string):
    a, b, c = input_string.split('-')
    return [float(a), float(b), float(c)]

In [4]:
def leave_one_out_grid(pairs, dropped_pair, experiment, combination, df, cf, cota_version):
    
    omega = {}
    for pair in pairs:
        omega[pair.iota_base] = pair.iota_abst
    
    hold_pairs = pairs.copy()
   
    if dropped_pair != None:
        hold_pairs.remove(dropped_pair)
        hold_omega  = ut.drop1omega(omega, dropped_pair.iota_base)
        
    else:
        hold_omega = omega
        
    I_relevant  = list(hold_omega.keys())

    struc, tree = ut.build_poset(I_relevant)
    chains      = ut.to_chains(hold_pairs, struc)
    
    combin = parse_comb(combination)
    kk, ll, mm = combin[0], combin[1], combin[2]
    
    args   = [hold_pairs, [chains], kk, ll, mm, df, cf]
    
    if cota_version == 'avg_plan':
        get_results.results_grid_looo(args, experiment, dropped_pair)
        looo_grid_results = ut.load_grid_results_looo(experiment, combination, dropped_pair)
        
    elif cota_version == 'avg_map':
        get_results.results_grid_looo_aggregated(args, experiment, dropped_pair)
        looo_grid_results = ut.load_grid_results_looo_aggregated(experiment, combination, dropped_pair)
    
    return looo_grid_results

def compute_ae_grid_looo(exp, pairs, combo, metric, cost, cota_version):
    
    looo_results_grid = []
    for n in range(len(pairs)):

        results_grid = {}
        for i, pair in enumerate(pairs[n]):

            if pair.iota_base.intervention != {None: None}:
                dropped_pair = pair
            else:
                dropped_pair = None

            results_grid[dropped_pair] = leave_one_out_grid(pairs[n], dropped_pair, exp, combo, metric, cost, cota_version)

        looo_results_grid.append(results_grid)
    
    return looo_results_grid

In [91]:
exp      = experiments[7]
pairs    = ut.load_pairs(exp)

metric   = 'fro'
cost     = 'Omega'

error    = 'jsd'
version  = 'avg_plan'
combo    = '0.2-0.5-0.3'

track_MI = False

In [92]:
def convert_matrix_lines_to_strings(matrix):
    lines_as_strings = []
    for line in matrix:
        line_string = ''.join([str(int(element)) for element in line])
        lines_as_strings.append(line_string)
    return lines_as_strings

def convert_list_to_matrix(strings_list):
    matrix = []
    for string in strings_list:
        row = [float(char) for char in string]
        matrix.append(row)
    return np.array(matrix)

def map_strings_to_values(strings_list, mapping_dict):
    string_counts = Counter(strings_list)
    new_dict = {}
    for string in strings_list:
        if string in mapping_dict:
            value = mapping_dict[string]
            count = string_counts[string]
            new_dict[value] = [value] * count
            
    
    res = [value for sublist in new_dict.values() for value in sublist]
    return res

In [107]:
def make_deter(tau, labels):# Initialize a dictionary to store the mappings of keys to labels with the highest probabilities
    highest_prob_mappings = {}

    # Iterate through each key in the stochastic mapping dictionary
    for key, probabilities in tau.items():
        # Check if the probability vector is the zero-vector
        if all(p == 0.0 for p in probabilities):
            # Choose a random index to select a label from l2
            random_index = random.randint(0, len(labels) - 1)

            # Get the corresponding label from l2 using the random index
            corresponding_label = labels[random_index]
        else:
            # Find the index of the maximum probability
            max_prob_index = probabilities.index(max(probabilities))

            # Get the corresponding label from l2 using the index
            corresponding_label = labels[max_prob_index]

        # Store the mapping in the result dictionary
        highest_prob_mappings[key] = corresponding_label

    return highest_prob_mappings

In [94]:
def make_deter(tau, labels):# Initialize a dictionary to store the mappings of keys to labels with the highest probabilities
    highest_prob_mappings = {}

    # Iterate through each key in the stochastic mapping dictionary
    for key, probabilities in tau.items():
        # Check if the probability vector is the zero-vector
        if all(p == 0.0 for p in probabilities):
            min_dist  = np.inf
            min_label = labels[random.randint(0, len(labels) - 1)]
            for i, label in enumerate(labels):
                dist = cts.hamming_distance(key, label)
                
                if dist<min_dist:
                    min_dist = dist
                    min_indx = i
                    
            # Get the corresponding label from l2 using the random index
            corresponding_label = labels[min_indx]
        else:
            # Find the index of the maximum probability
            max_prob_index = probabilities.index(max(probabilities))

            # Get the corresponding label from l2 using the index
            corresponding_label = labels[max_prob_index]

        # Store the mapping in the result dictionary
        highest_prob_mappings[key] = corresponding_label

    return highest_prob_mappings

In [108]:
df_base = joblib.load('data/battery_discrete/df_baseB.pkl')
df_abst = joblib.load('data/battery_discrete/df_abstB.pkl')

In [109]:
labels = []
labels.append(pairs[0][0].base_labels)
labels.append(pairs[0][0].abst_labels)

In [110]:
res = compute_ae_grid_looo(exp, pairs, combo, metric, cost, version)[0]

In [111]:
sts       = convert_matrix_lines_to_strings(df_base.values)
tau_detrm = make_deter(res[None][0], labels[1])

formatted_tau_dict = {
    ''.join(map(str, key)): ''.join(map(str, value)) for key, value in tau_detrm.items()
}
tau_samples = convert_list_to_matrix(map_strings_to_values(sts, formatted_tau_dict))

In [112]:
tau_df = pd.DataFrame(tau_samples, columns=df_abst.columns)

In [113]:
enhanced_abst = pd.concat([df_abst, tau_df], ignore_index=True) #enhanced_abst

In [114]:
swap = {0: 75, 1: 100, 2: 200}

df_abst['Comma gap (µm)'] = df_abst['Comma gap (µm)'].replace(swap)
enhanced_abst = pd.concat([df_abst, tau_df], ignore_index=True)
enhanced_abst['Comma gap (µm)'] = enhanced_abst['Comma gap (µm)'].replace(swap)

In [115]:
n_CG_LRCS = len(df_abst['Comma gap (µm)'].unique())
n_ML_LRCS = 5
dom_CG_LRCS = list(set(df_abst['Comma gap (µm)'].values)) # [75,100,200]
dom_ML_LRCS = np.arange(n_ML_LRCS)

In [116]:
def eval_metric(model,Xte,yte,Xout,roundpred=True):
    if roundpred:
        preds = np.round(model.predict(Xte.reshape(-1,1)))
    else:
        preds = model.predict(Xte.reshape(-1,1))
    mses = (preds - yte)**2
    print('MSE (with {0} out): {1} ({2})'.format(Xout,np.mean(mses),np.std(mses)))
    print('${0:.2f}\pm{1:.2f}$'.format(np.mean(mses),np.std(mses)))
    
    return np.mean(mses)

def select_one_X_out(X,y,cond):
    Xte = np.array(X[cond])
    yte = np.array(y[cond])

    Xtr = np.array(X[np.logical_not(cond)])
    ytr = np.array(y[np.logical_not(cond)])
    
    return Xtr,ytr,Xte,yte

## Before abstraction

In [117]:
#Lasso
scores_a = []

for cg in dom_CG_LRCS:
    cond = df_abst['Comma gap (µm)']==cg
    
    Xtr,ytr,Xte,yte = select_one_X_out(df_abst['Comma gap (µm)'],df_abst['binned ML'],cond)
    model = Lasso().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,False)
    scores_a.append(score)
    
print('Overall MSE: {0} ({1})'.format(np.mean(scores_a),np.std(scores_a)))
print('${0:.2f}\pm{1:.2f}$'.format(np.mean(scores_a),np.std(scores_a)))

MSE (with 200 out): 3.358400000000001 (0.9112079894294169)
$3.36\pm0.91$
MSE (with 75 out): 0.6362090277777774 (0.7281328087325675)
$0.64\pm0.73$
MSE (with 100 out): 0.21659716000000065 (0.0)
$0.22\pm0.00$
Overall MSE: 1.4037353959259262 (1.392732043074578)
$1.40\pm1.39$


## After abstraction

Learning on LRCS+WMG data: one-X out with WMG providing the missing support

In [118]:
scores_b = []

for cg in dom_CG_LRCS:
    cond0 = df_abst['Comma gap (µm)']==cg
    cond1 = list(cond0) + [False]*(len(enhanced_abst)-len(cond0))
    
    Xtr,ytr,Xte,yte = select_one_X_out(enhanced_abst['Comma gap (µm)'],enhanced_abst['binned ML'],cond1)
    
    model = Lasso().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,False)
    scores_b.append(score)
    
print('Overall MSE: {0} ({1})'.format(np.mean(scores_b),np.std(scores_b)))
print('${0:.2f}\pm{1:.2f}$'.format(np.mean(scores_b),np.std(scores_b)))

MSE (with 200 out): 0.04152396170310105 (0.16722099024356046)
$0.04\pm0.17$
MSE (with 75 out): 0.2503504775522152 (0.49638727062556837)
$0.25\pm0.50$
MSE (with 100 out): 0.08763420978073998 (0.0)
$0.09\pm0.00$
Overall MSE: 0.12650288301201873 (0.08957383825459315)
$0.13\pm0.09$


Learning on LRCS+WMG data: one X-out with WMG not providing the missing support

In [119]:
scores_c = []

for cg in dom_CG_LRCS:
    cond = enhanced_abst['Comma gap (µm)']==cg
    
    Xtr,ytr,Xte,yte = select_one_X_out(enhanced_abst['Comma gap (µm)'],enhanced_abst['binned ML'],cond)
    
    model = Lasso().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,False)
    scores_c.append(score)
    
print('Overall MSE: {0} ({1})'.format(np.mean(scores_c),np.std(scores_c)))
print('${0:.2f}\pm{1:.2f}$'.format(np.mean(scores_c),np.std(scores_c)))

MSE (with 200 out): 0.6897503886465074 (0.9111193121851174)
$0.69\pm0.91$
MSE (with 75 out): 0.671449079188327 (0.8063169776084173)
$0.67\pm0.81$
MSE (with 100 out): 0.33602740343770643 (0.6617402485215056)
$0.34\pm0.66$
Overall MSE: 0.5657422904241803 (0.1626046974708446)
$0.57\pm0.16$
